<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/static/images/logo-v4.svg" width=190/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/develop/examples/model_assisted_labeling/video_mal.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/tree/develop/examples/model_assisted_labeling/video_mal.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# Video MAL

* Upload model inferences for video tasks
* Support types
    * bounding box

In [ ]:
!pip install -q 'labelbox[data]'

In [2]:
import os
import uuid
from io import BytesIO
from typing import Dict, Any, Tuple

from labelbox import Client, LabelingFrontend, MediaType, MALPredictionImport, LabelImport
from labelbox.schema.ontology import OntologyBuilder, Tool, Classification, Option
from labelbox.schema.queue_mode import QueueMode
from labelbox.data.annotation_types import (
    Label, TextData, Checklist, Radio, ObjectAnnotation, TextEntity,
    ClassificationAnnotation, ClassificationAnswer, LabelList
)

# API Key and Client
Provide a valid api key below in order to properly connect to the Labelbox Client.

In [3]:
# Add your api key
API_KEY=None
client = Client(api_key=API_KEY)

### Project Setup

In [4]:

# Project defaults to batch mode with benchmark quality settings if this argument is not provided
# Queue mode will be deprecated once dataset mode is deprecated
mal_project = client.create_project(name="video_mal_project_demo",
                                    queue_mode=QueueMode.Batch,
                                    auto_audit_percentage=1,
                                    auto_audit_number_of_labels=1,
                                    media_type=MediaType.Video)



li_project = client.create_project(name="video_label_import_project_demo",
                                    queue_mode=QueueMode.Batch,
                                    auto_audit_percentage=1,
                                    auto_audit_number_of_labels=1,
                                    media_type=MediaType.Video)

# # Create one Labelbox dataset
dataset = client.create_dataset(name="video_annotation_import_demo_dataset")
uploads = {
    "row_data":"https://storage.googleapis.com/labelbox-datasets/video-sample-data/sample-video-2.mp4",
    "global_key": "TEST-ID-%id" % uuid.uuid1()
}

data_row = dataset.create_data_row(uploads)
print(data_row)


######################### DATASET CONSENSUS OPTION ########################
#Note that dataset base projects will be deprecated in the near future.

#To use Datasets/Consensus instead of Batches/Benchmarks use the following query: 
#In this case, 10% of all data rows need to be annotated by three labelers.

# dataset_project = client.create_project(name="dataset-test-project",
#                                 description="a description",
#                                 media_type=MediaType.Video,
#                                 auto_audit_percentage=0.1,
#                                 auto_audit_number_of_labels=3,
#                                 queue_mode=QueueMode.Dataset)

# dataset_project.datasets.connect(dataset)


<DataRow {'created_at': datetime.datetime(2022, 10, 28, 13, 57, tzinfo=datetime.timezone.utc), 'external_id': None, 'global_key': 'TEST-ID-133625741652964826529256970019634151426d', 'media_attributes': {}, 'metadata': [], 'metadata_fields': [], 'row_data': 'https://storage.googleapis.com/labelbox-datasets/video-sample-data/sample-video-2.mp4', 'uid': 'cl9sk660t5tki08zs35957mia', 'updated_at': datetime.datetime(2022, 10, 28, 13, 57, tzinfo=datetime.timezone.utc)}>


In [5]:
# We need the data row ID to create a batch
batch_datarows = [dr.uid for dr in list(dataset.export_data_rows())]

# Create a batch to send to your MAL project
batch_mal = mal_project.create_batch(
  "video-batch-MAL-demo", # Each batch in a project must have a unique name
  batch_datarows, # A list of data rows or data row ids
  5 # priority between 1(Highest) - 5(lowest)
)

# Create a batch to send to you LIM project
batch_li = li_project.create_batch(
    "video-batch-LIM-demo", # Each batch in a project must have a unique name
    batch_datarows, # A list of data rows or data row ids
    5 # priority between 1(Highest) - 5(lowest)
)

# Setup your ontology / labeling editor
ontology_builder = OntologyBuilder(
    tools=[Tool(tool=Tool.Type.BBOX, name="jellyfish")])

editor = next(client.get_labeling_frontends(where=LabelingFrontend.name == "Editor")) # Unless using a custom editor,
# Connect your ontology and editor to your MAL and LI project
mal_project.setup(editor, ontology_builder.asdict())
li_project.setup(editor, ontology_builder.asdict())

print("Batch Li: ", batch_li)
print("Batch Mal: ", batch_mal)

Batch Li:  <Batch {'created_at': datetime.datetime(2022, 10, 28, 13, 57, 9, tzinfo=datetime.timezone.utc), 'name': 'video-batch-LIM-demo', 'size': 1, 'uid': '6a7fed10-56c8-11ed-a20f-291accbaa64c', 'updated_at': datetime.datetime(2022, 10, 28, 13, 57, 9, tzinfo=datetime.timezone.utc)}>
Batch Mal:  <Batch {'created_at': datetime.datetime(2022, 10, 28, 13, 57, 8, tzinfo=datetime.timezone.utc), 'name': 'video-batch-MAL-demo', 'size': 1, 'uid': '699c43d0-56c8-11ed-95ed-7b8551165232', 'updated_at': datetime.datetime(2022, 10, 28, 13, 57, 8, tzinfo=datetime.timezone.utc)}>


#### Grab featureSchemaIds

In [6]:
# When we created a project with the ontology defined above, all of the ids were assigned.
# So lets reconstruct the ontology builder with all of the ids.
ontology_li = ontology_builder.from_project(li_project)
ontology_mal = ontology_builder.from_project(mal_project)
# # We want all of the feature schemas to be easily accessible by name.
schema_lookup_li = {tool.name: tool.feature_schema_id for tool in ontology_li.tools}
schema_lookup_mal = {tool.name: tool.feature_schema_id for tool in ontology_mal.tools}

print(schema_lookup_li)
print(schema_lookup_mal)

{'jellyfish': 'cl9sk6g3nd33j07xy52vz1noq'}
{'jellyfish': 'cl9sk6ehad32w07xyhm8f96xw'}


## Import Format

**segments**: A segment represents a continuous section where an object is visible. If an instance disappears then the segment ends. If it re-appears, a new segment is created.

**keyframes**: Key frames identify the location of an instance. Between keyframes, the location of the instance is interpolated.

**bbox**: The coordinates of the bounding box

In [7]:
segments = [{
    "keyframes": [{
        "frame": 1,
        "bbox": {
            "top": 80,
            "left": 80,
            "height": 80,
            "width": 80
        }
    }, {
        "frame": 20,
        "bbox": {
            "top": 125,
            "left": 125,
            "height": 200,
            "width": 300
        }
    }]
}, {
    "keyframes": [{
        "frame": 27,
        "bbox": {
            "top": 80,
            "left": 50,
            "height": 80,
            "width": 50
        }
    }]
}]

##### Create helper functions to make this much easier

In [9]:
def create_video_bbox_ndjson(datarow_id: str, schema_id: str,
                             segments: Dict[str, Any]) -> Dict[str, Any]:
    return {
        "uuid": str(uuid.uuid4()),
        "schemaId": schema_id,
        "dataRow": {
            "id": datarow_id
        },
        "segments": segments
    }

In [10]:
uploads_li = []
uploads_mal = []


for data_row in dataset.data_rows():
    uploads_li.append(
        create_video_bbox_ndjson(data_row.uid, schema_lookup_li['jellyfish'],
                                 segments))
    
for data_row in dataset.data_rows():
    uploads_mal.append(
        create_video_bbox_ndjson(data_row.uid, schema_lookup_mal['jellyfish'],
                                 segments))

### Upload the annotations with MAL import

In [11]:
# Let's upload!
# Validate must be set to false for video bounding boxes
upload_job = MALPredictionImport.create_from_objects(
    client = client, 
    project_id = mal_project.uid, 
    name="MAL_upload_label_import_job_demo", 
    predictions=uploads_mal
)

In [12]:
# Wait for upload to finish (Will take up to five minutes)
upload_job.wait_until_done()
# Review the upload status
print("Errors: ",upload_job.errors)

Errors:  []


### Upload the annotations with LabelImport

In [13]:
upload_job_li = LabelImport.create_from_objects(
    client = client,
    project_id = li_project.uid, 
    name = "LI_upload_label_import_job_demo",
    labels=uploads_li
)

In [14]:
# Wait for upload to finish (Will take up to five minutes)
upload_job_li.wait_until_done()
# Review the upload status
print("Errors: ",upload_job_li.errors)

Errors:  []


## Cleanup

In [15]:
# li_project.delete()
# mal_project.delete()
# dataset.delete()